In [89]:
import torch
import torchvision.transforms  as T
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

device = torch.device('cuda')
device

device(type='cuda')

In [31]:
set = datasets.MNIST(root='./data', train=True, download=True, transform=T.ToTensor())
loader = DataLoader(set, batch_size = 64, shuffle=False)

In [32]:
def get_mean_std(loader):
    channels_sum, channels_squares_sum = 0, 0
    num_batches = len(loader)

    for data, _ in loader:
        channels_sum += torch.mean(data)
        channels_squares_sum += torch.mean(data**2)

    mean = channels_sum/num_batches
    std  = np.sqrt((channels_squares_sum/num_batches - mean**2))

    return mean, std

mean, std = get_mean_std(loader)
print(mean, std)

tensor(0.1307) tensor(0.3081)


In [117]:
trans = T.Compose([T.ToTensor(), T.Normalize(mean.item(), std.item())])

trainset = datasets.MNIST(root='./data', train=True, download=True, transform=trans)
train_loader = DataLoader(trainset, batch_size = 64, shuffle=False)

testset = datasets.MNIST(root='./data', train=False, download=True, transform=trans)
test_loader = DataLoader(testset, batch_size = 64, shuffle=False)

In [118]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.featuresCONV2D = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.Dropout(0.2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.Dropout(0.2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.Dropout(0.2),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.Dropout(0.2),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifierLINEAR = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 1024),
            nn.Dropout(0.2),
            nn.LeakyReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.Dropout(0.2),
            nn.LeakyReLU(inplace=True),
            nn.Linear(512, 256),
            nn.Dropout(0.2),
            nn.LeakyReLU(inplace=True),
            nn.Linear(256, 128),
            nn.Dropout(0.2),
            nn.LeakyReLU(inplace=True),
            nn.Linear(128, 64),
            nn.Dropout(0.2),
            nn.LeakyReLU(inplace=True),
            nn.Linear(64, 32),
            nn.Dropout(0.2),
            nn.LeakyReLU(inplace=True),
            nn.Linear(32, 10)
            )

    def forward(self, x):
        x = self.featuresCONV2D(x)
        x = self.classifierLINEAR(x)
        
        return x   

In [119]:
model = Net().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [120]:
model.train()

for epoch in range(25):
    running_loss = 0.0
    correct = 0
    total = 0
    losses = []

    for i, (img, label) in enumerate(train_loader):
        img, label = img.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(img)
        
        loss = criterion(output, label)
        loss.backward()

        optimizer.step()

        _, predicted = torch.max(output.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
        running_loss += loss.item()
        losses.append(loss.item())
        
    print(f'Epoch {epoch+1} = loss: {running_loss / len(train_loader):.3f} | accuracy: {100 * correct // total}%')

print('Training finished!')        

Epoch 1 = loss: 0.672 | accuracy: 76%
Epoch 2 = loss: 0.144 | accuracy: 96%
Epoch 3 = loss: 0.099 | accuracy: 97%
Epoch 4 = loss: 0.077 | accuracy: 98%
Epoch 5 = loss: 0.063 | accuracy: 98%
Epoch 6 = loss: 0.054 | accuracy: 98%
Epoch 7 = loss: 0.049 | accuracy: 98%
Epoch 8 = loss: 0.044 | accuracy: 98%
Epoch 9 = loss: 0.042 | accuracy: 99%
Epoch 10 = loss: 0.034 | accuracy: 99%
Epoch 11 = loss: 0.034 | accuracy: 99%
Epoch 12 = loss: 0.032 | accuracy: 99%
Epoch 13 = loss: 0.027 | accuracy: 99%
Epoch 14 = loss: 0.027 | accuracy: 99%
Epoch 15 = loss: 0.025 | accuracy: 99%
Epoch 16 = loss: 0.024 | accuracy: 99%
Epoch 17 = loss: 0.021 | accuracy: 99%
Epoch 18 = loss: 0.022 | accuracy: 99%
Epoch 19 = loss: 0.020 | accuracy: 99%
Epoch 20 = loss: 0.018 | accuracy: 99%
Epoch 21 = loss: 0.018 | accuracy: 99%
Epoch 22 = loss: 0.016 | accuracy: 99%
Epoch 23 = loss: 0.016 | accuracy: 99%
Epoch 24 = loss: 0.016 | accuracy: 99%
Epoch 25 = loss: 0.014 | accuracy: 99%
Training finished!


In [121]:
correct = 0
total = 0
with torch.no_grad():
    for img, label in test_loader:
        images, labels = img.to(device), label.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {len(test_loader)} test images: {100 * correct // total}%')

Accuracy of the network on the 157 test images: 99%


In [122]:
torch.save(model.state_dict(), '/media/braddock/LInux/MNIST/model.pt')

In [123]:
import pandas as pd

In [124]:
model = Net().to(device)
model.load_state_dict(torch.load("model.pt")) #Score: 0.9969
model.eval()

Net(
  (featuresCONV2D): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Dropout(p=0.2, inplace=False)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): LeakyReLU(negative_slope=0.01, inplace=True)
    (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): Dropout(p=0.2, inplace=False)
    (6): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.01, inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Dropout(p=0.2, inplace=False)
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): LeakyReLU(negative_slope=0.01, inplace=True)
    (13): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): Dropout(p=0.2, inpla

In [125]:
class testData(Dataset):
    def __init__(self, x):
        self.x = x
    
    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        return self.x[index]

In [126]:
test = torch.tensor(np.array(pd.read_csv('test.csv')).reshape(-1, 1, 28, 28), dtype=torch.float32)
testset = testData(test)
test_loader2 = DataLoader(testset, batch_size = 64, shuffle=False)

In [131]:
test_pred = torch.LongTensor()

with torch.no_grad():
  for data in test_loader2:
      data = data.to(device)
          
      output = model(data)
      
      pred = output.cpu().data.max(1, keepdim=True)[1]
      test_pred = torch.cat((test_pred, pred), dim=0)

In [132]:
test_pred

tensor([[2],
        [0],
        [9],
        ...,
        [3],
        [9],
        [2]])

In [133]:
out_df = pd.DataFrame(np.c_[np.arange(1, len(test)+1)[:,None], test_pred.numpy()], columns=['ImageId', 'Label'])

In [134]:
out_df.to_csv('my_submmision.csv', index=False) #Score: 0.99696